In [1]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import time
import itertools
import pysam
from Bio import SeqIO, AlignIO
from Bio.Seq import reverse_complement
from Bio.Align import AlignInfo
import edlib
import numpy as np
import markov_clustering as mc
from tqdm import tqdm
from sklearn.manifold import TSNE, MDS
from bokeh.plotting import figure, ColumnDataSource, show
from bokeh.palettes import viridis, magma, plasma
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
def make_distance_matrix(sequences):
    dists = np.array([np.array([0 for _ in range(len(sequences))]) for _ in range(len(sequences))])
    
    base_seq = sequences[0]["seq"]
    for s in tqdm(sequences, desc="{:<10}".format("prescan")):
        fwd = s["seq"]
        rev = s["revseq"]
        d_fwd = edlib.align(fwd, base_seq, task="distance", mode="NW")["editDistance"]
        d_rev = edlib.align(rev, base_seq, task="distance", mode="NW")["editDistance"]
        if d_fwd > d_rev:
            s["dir"] = 1
        else:
            s["dir"] = -1

    for i in tqdm(range(len(dists)), desc="{:<10}".format("align")):
        query = sequences[i]["seq"] if sequences[i]["dir"] == 1 else sequences[i]["revseq"]
    
        for j in range(i, len(dists[i])):
            if i != j:
                target = sequences[j]["seq"] if sequences[j]["dir"] == 1 else sequences[j]["revseq"]
                d = edlib.align(query, target, task="distance", mode="NW")["editDistance"]
                dists[i][j] = d
                dists[j][i] = d
    return dists

def make_similarity_from_dists(dists):
    max_dist = np.max(dists)
    return 1.0 - dists / max_dist

def euclidean_dist(p1, p2):
    return np.sqrt(np.power(p1[0] - p2[0], 2) + np.power(p1[1] - p2[1], 2))

def mask_homopolymers(sequence, max_length=2):
    masked = []
    
    def base_counter():
        return {"nuc": "", "count": 0}
    
    current = base_counter()
    for base in sequence:
        if base != current["nuc"]:
            masked.append(current["nuc"] * min(max_length, current["count"]))
            current = base_counter()
        
        current["nuc"] = base
        current["count"] += 1
    
    masked.append(current["nuc"] * min(max_length, current["count"]))
    
    return "".join(masked)
            

In [122]:
plate = "plate3"
barcode = "lbc49"
ccs_bam = "/exports/humgen/wgallard/pharmacogenomics_pipeline/pugwash/LIMA_2_stage_deflate/{}/preprocessor/CCS/{}.bam".format(plate, barcode)
subreads_bam = "/exports/humgen/wgallard/pharmacogenomics_pipeline/pugwash/LIMA_2_stage_deflate/{}/preprocessor/consolidated/{}.bam".format(plate, barcode)
genome = "/bam-export/Guy/GRCh38_no_alt_analysis_set/sequence/GRCh38_no_alt_analysis_set.fasta"

work_dir = "/exports/humgen/wgallard/CCS_Cluster/masked/{}_{}".format(plate, barcode)
!mkdir -p {work_dir}
os.chdir(work_dir)

In [123]:
# read from ccs bam
with pysam.AlignmentFile(ccs_bam, "rb", check_sq=False) as infile:
    sequences = [
        {"id": r.query_name, 
         "seq": mask_homopolymers(r.query_sequence), 
         "revseq": mask_homopolymers(reverse_complement(r.query_sequence)), 
         "passes": r.get_tag("np"),
         "qual": r.get_tag("rq"),
         "len": r.query_length}
         for r in list(infile.fetch(until_eof=True))]

print(len(sequences), "ccs sequences")

min_passes = 1
min_length = 6000
max_length = 7000
min_qual = 0.98
hq_sequences = [s for s in sequences if min_passes <= s["passes"] and min_length <= s["len"] <= max_length and s["qual"] >= min_qual]
print("{} high qual sequences: passes >= {}; {} <= length <= {}; qual >= {}".format(len(hq_sequences), min_passes, min_length, max_length, min_qual))

232 ccs sequences
118 high qual sequences: passes >= 1; 6000 <= length <= 7000; qual >= 0.98


In [ ]:
# Note
# the above is equivalent to
# bamtools filter -in {ccs_bam} -out ccs.filtered.bam -length '>=6000' -length '<=7000' -tag 'np:>=1' -tag 'rq:>=0.98'
# pbindex ccs.filtered.bam
# bam2fasta -o - ccs.filtered.bam > ccs.filtered.fasta
# fastools 

In [124]:
dists = make_distance_matrix(hq_sequences)

align     : 100%|██████████| 118/118 [00:02<00:00, 42.51it/s]


In [125]:
tsne=TSNE(n_components=2, metric="precomputed", 
          n_iter=5000, learning_rate=50, 
          #perplexity=int(len(hq_sequences) / 10), 
          method="exact", verbose=1)
embedded = tsne.fit_transform(dists)

datasource = ColumnDataSource({
    "x": [e[0] for e in embedded], 
    "y": [e[1] for e in embedded], 
    "passes": [s["passes"] for s in hq_sequences], 
    "seqid": [s["id"] for s in hq_sequences],
})

TOOLTIPS = [
    ("Sequence", "@seqid"),
    ("Passes", "@passes"),
    ("index", "$index")
]

p=figure(tooltips=TOOLTIPS)
p.circle(x="x", y="y", size="passes",source=datasource)
show(p)

[t-SNE] Computed conditional probabilities for sample 118 / 118
[t-SNE] Mean sigma: 2.222573
[t-SNE] KL divergence after 250 iterations with early exaggeration: 85.986037
[t-SNE] KL divergence after 1250 iterations: 0.639466


In [126]:
dists_2d = []
for p1 in embedded:
    row = []
    for p2 in embedded:
        dist = euclidean_dist(p1, p2)
        row.append(dist)
    dists_2d.append(row)

similarity = make_similarity_from_dists(dists_2d)
similarity[similarity < np.percentile(similarity, 80)] = 0

results = mc.run_mcl(similarity, inflation=1.4)
clusters = sorted(mc.get_clusters(results), key=lambda x: len(x), reverse=True)

scraps = [c for c in clusters if len(c) <= 5]

clusters = [c for c in clusters if len(c) > 5] + [tuple(itertools.chain(*scraps))]

for i, c in enumerate(clusters):
    print("cluster {}:".format(i), len(c))

cluster_map = {node: i for i, cluster in enumerate(clusters) for node in cluster}

datasource = ColumnDataSource({
    "x": [e[0] for e in embedded], 
    "y": [e[1] for e in embedded], 
    "passes": [s["passes"] for s in hq_sequences], 
    "seqid": [s["id"][-20:] for s in hq_sequences],
    "cluster": [str(cluster_map[i]) for i in range(dists.shape[0])]
})

TOOLTIPS = [
    ("Sequence", "@seqid"),
    ("Passes", "@passes"),
    ("cluster", "@cluster"),
    ("index", "$index")
]

factors = [str(i) for i in range(len(clusters))]


p=figure(tooltips=TOOLTIPS)
p.circle(x="x", y="y", size="passes", source=datasource, color=factor_cmap('cluster', palette=viridis(len(clusters)), factors=factors))
show(p)

cluster 0: 106
cluster 1: 7
cluster 2: 5


In [127]:
p = figure()
top_cluster = len(clusters[0])
cluster_sizes = [len(c) for c in clusters]
proportions = [c / top_cluster for c in cluster_sizes]
p.line(x=list(range(len(clusters))), y=proportions, line_width=3)
show(p)
print(proportions)

[1.0, 0.0660377358490566, 0.04716981132075472]


In [138]:
def dump_cluster_seqs_to_bam(clusters, cluster_map, sequences, aligned_ccs_bam):
    # map sequence_id to cluster
    sequence_to_cluster = {}
    for i, record in enumerate(sequences):
        try:
            sequence_to_cluster[record["id"]] = cluster_map[i]
        except KeyError:
            pass

    with pysam.AlignmentFile(aligned_ccs_bam, "rb", check_sq=False) as infile:
        out_bams = [pysam.AlignmentFile("{}_{}_cluster{}.bam".format(plate, barcode, c), "wb", template=infile) for c in range(len(clusters))]
    
        for read in infile.fetch(until_eof=True):
            qname = read.query_name
            try:
                cluster = sequence_to_cluster[qname]
            except KeyError:
                continue
            out_bams[cluster].write(read)
    
    for out_bam in out_bams:
        out_bam.close()


def dump_cluster_seqs_to_fasta(clusters, cluster_map, sequences):
    # map sequence_id to cluster
    sequence_to_cluster = {}
    for i, record in enumerate(sequences):
        try:
            sequence_to_cluster[record["id"]] = cluster_map[i]
        except KeyError:
            pass

    out_files = [open("{}_{}_cluster{}.fasta".format(plate, barcode, c), "w") for c in range(len(clusters))]
    
    for read in sequences:
        qname = read["id"]
        seq = read["seq"]
        strand = read["dir"]
        if strand != 1:
            seq = reverse_complement(seq)
        try:
            cluster = sequence_to_cluster[qname]
        except KeyError:
            continue
        print(">{}\n{}".format(qname, seq), file=out_files[cluster])
    
    for out_file in out_files:
        out_file.close()


def dump_whitelist(clusters, cluster_map, sequences, whitelist_path):
    # map sequence_id to cluster
    sequence_to_cluster = {}
    for i, record in enumerate(sequences):
        try:
            sequence_to_cluster[record["id"]] = cluster_map[i]
        except KeyError:
            pass
    
    with open(whitelist_path, "w") as whitelist:
        for sequence in sequences:
            seq_id = sequence["id"]
            try:
                cluster = sequence_to_cluster[seq_id]
            except KeyError:
                continue
            whitelist.write("/".join(seq_id.split("/")[:-1]) + "\n")


def tag_cluster_sequences(clusters, sequences):
    for i,cluster in enumerate(clusters):
        for seq_index in cluster:
            sequences[seq_index]["cluster"] = i


def wait_for_jobs(job_ids, recheck_wait=5):
    running_jobs = set(job_ids)
    while len(running_jobs) > 0:
        finished = set()
        for j in running_jobs:
            state = !qstat -j {j}
            if state[0] == "Following jobs do not exist: ":
                print("job {} finished".format(j), flush=True)
                finished.add(j)
    
        running_jobs -= finished
        time.sleep(recheck_wait)

def run_cluster_jobs(joblist):
    job_ids = []
    for job in joblist:
        job_id = !qsub -terse {job}
        job_ids.append(job_id)

    job_ids = list(itertools.chain.from_iterable(job_ids))
    print("submitted job ids", job_ids, flush=True)
    wait_for_jobs(job_ids)

In [129]:
filtered_clusters = []
for cluster in clusters:
    # prioritize by distance from cluster center and number of passes
    cluster_center = [np.mean([embedded[c][0] for c in cluster]), np.mean([embedded[c][1] for c in cluster])]
    offset = [euclidean_dist(cluster_center, embedded[x]) for x in range(len(embedded))]
    passes = [s["passes"] for s in hq_sequences]
    assert len(passes) == len(offset)
    cluster_sorted = sorted(cluster, key=lambda x: (-offset[x], passes[x]), reverse=True)
    cluster_passes = [passes[c] for c in cluster_sorted]
    cluster_filtered = [c for i,c in enumerate(cluster_sorted) if sum(cluster_passes[:i]) < 500]
    filtered_clusters.append(cluster_filtered)

top_clusters = []
for i, p in enumerate(proportions):
    if p > 0.4:
        top_clusters.append(filtered_clusters[i])

filtered_cluster_map = {node: i for i, cluster in enumerate(top_clusters) for node in cluster}
print(len(top_clusters), "clusters selected")
tag_cluster_sequences(filtered_clusters, hq_sequences)

datasource = {"x": [], "y": [], "passes": [], "seqid":[], "cluster": [], "included": []}

for i, cluster in enumerate(clusters):
    try:
        tc = top_clusters[i]
    except IndexError:
        tc = ()
    for seq_index in clusters[i]:
        datasource["x"].append(embedded[seq_index][0])
        datasource["y"].append(embedded[seq_index][1])
        datasource["passes"].append(hq_sequences[seq_index]["passes"])
        datasource["seqid"].append(hq_sequences[seq_index]["id"][-20:])
        datasource["cluster"].append(str(i))
        datasource["included"].append(1.0 if seq_index in tc else 0.2)

datasource = ColumnDataSource(datasource)

TOOLTIPS = [
    ("Sequence", "@seqid"),
    ("Passes", "@passes"),
    ("cluster", "@cluster"),
    ("index", "$index")
]

factors = [str(i) for i in range(len(clusters))]


p=figure(tooltips=TOOLTIPS)
p.circle(x="x", y="y", size="passes", alpha="included", source=datasource, color=factor_cmap('cluster', palette=viridis(len(clusters)), factors=factors))
show(p)

1 clusters selected


In [30]:
#dump_cluster_seqs_to_bam(top_clusters, filtered_cluster_map, hq_sequences, ccs_bam)

In [130]:
dump_cluster_seqs_to_fasta(top_clusters, filtered_cluster_map, hq_sequences)

In [131]:
# ======================
# do an MSA on the fasta sequences
# ======================
for i in range(len(top_clusters)):
    print("generating MSA for cluster {}".format(i))
    !mafft --auto --quiet {plate}_{barcode}_cluster{i}.fasta > {plate}_{barcode}_cluster{i}.mafft

generating MSA for cluster 0


In [132]:
# then generate consensus for the clusters
for i in range(len(top_clusters)):
    msa = AlignIO.read("{p}_{b}_cluster{c}.mafft".format(p=plate, b=barcode, c=i), "fasta")

    summary_align = AlignInfo.SummaryInfo(msa)

    consensus = str(summary_align.gap_consensus(0.51, require_multiple=True, ambiguous='')).replace("-", "")
    with open("{p}_{b}_cluster{c}.consensus.fa".format(p=plate, b=barcode, c=i), "w") as outfile:
        print(">{}\n{}".format("consensus", consensus), file=outfile)

In [133]:
# +------------------+
# | PHASING WORKFLOW |
# +------------------+
for i in range(len(top_clusters)):
    prefix = "{p}_{b}_cluster{c}".format(p=plate, b=barcode, c=i)

    !ngmlr -r {prefix}.consensus.fa -q {prefix}.fasta --rg-id foo --rg-sm sm --rg-lb lb > {prefix}.aligned.sam
    !samtools sort {prefix}.aligned.sam > {prefix}.aligned.bam
    !samtools index {prefix}.aligned.bam
    !bcftools mpileup -f {prefix}.consensus.fa {prefix}.aligned.bam | bcftools call -mv -Ob -o {prefix}.bcf
    !bcftools view -i '%QUAL>=50' {prefix}.bcf > {prefix}.vcf
    !whatshap phase --indels {prefix}.vcf {prefix}.aligned.bam -o {prefix}.phased.vcf
    !bgzip {prefix}.phased.vcf
    !tabix {prefix}.phased.vcf.gz
    !whatshap haplotag -o {prefix}.aligned.tagged.bam {prefix}.phased.vcf.gz {prefix}.aligned.bam
    !samtools index {prefix}.aligned.tagged.bam

ngmlr 0.2.7 (build: Jul  2 2018 10:32:15, start: 2018-10-18.15:37:45)
Contact: philipp.rescheneder@univie.ac.at
Writing output (SAM) to stdout
Encoding reference sequence.
Size of reference genome 0 Mbp (max. 68719 Mbp)
0 reference sequences were skipped (length < 10).
Writing encoded reference to plate3_lbc49_cluster0.consensus.fa-enc.2.ngm
Writing to disk took 0.01s
Building reference index #0 (kmer length: 13, reference skip: 2)
0 prefixes were ignored due to the frequency cutoff (1000)
Overall time for creating RefTable: 3.35s
Writing reference index to plate3_lbc49_cluster0.consensus.fa-ht-13-2.2.ngm
Writing to disk took 3.22s
Opening query file plate3_lbc49_cluster0.fasta
Mapping reads...
Processed: 37 (1.00), R/S: 18.50, RL: 6512, Time: 2.75 1.50 82.00, Align: 1.00, 327, 1.00
Processed: 82 (1.00), R/S: 20.50, RL: 6024, Time: 3.00 1.49 93.81, Align: 1.00, 319, 1.00
Processed: 82 (1.00), R/S: 20.50, RL: 6024, Time: 3.00 1.49 93.81, Align: 1.00, 319, 1.00
Done (82 reads mapped (100

In [134]:
# ensure that single heterozygous variants are assigned a phase
for i in range(len(top_clusters)):
    with pysam.VariantFile("{p}_{bc}_cluster{c}.phased.vcf.gz".format(p=plate, bc=barcode, c=i)) as vcf_in:
        header = vcf_in.header
        records = list(r for r in vcf_in.fetch())

    if len(records) == 1:
        record = records[0]
        if not "PS" in header.formats:
            header.formats.add("PS",1,"Integer","Phase set identifier")
        sample = record.samples["sm"]
        genotype = sample["GT"]
        if not sample.phased and genotype[0] != genotype[1]: # heterozygous
            sample["PS"] = record.pos
            sample.phased = True

            with pysam.VariantFile("{p}_{bc}_cluster{c}.phased.vcf".format(p=plate, bc=barcode, c=i), 'w', header=header) as vcf_out:
                vcf_out.write(record)
            
            !bgzip -f {plate}_{barcode}_cluster{i}.phased.vcf
            !tabix {plate}_{barcode}_cluster{i}.phased.vcf.gz
            !whatshap haplotag -o {plate}_{barcode}_cluster{i}.aligned.tagged.bam {plate}_{barcode}_cluster{i}.phased.vcf.gz {plate}_{barcode}_cluster{i}.aligned.bam
            !samtools index {plate}_{barcode}_cluster{i}.aligned.tagged.bam

In [135]:
# generate whitelist for LAA consensus
for i in range(len(top_clusters)):
    tagged_reads = {}
    with pysam.AlignmentFile("{p}_{bc}_cluster{c}.aligned.tagged.bam".format(p=plate, bc=barcode, c=i), "rb", check_sq=False) as infile:
        for r in infile.fetch(until_eof=True):
            qname = r.query_name
            molecule = "/".join(qname.split("/")[:-1])
            try:
                haplotype = r.get_tag("HP")
            except KeyError:
                haplotype = 0

            if haplotype not in tagged_reads:
                tagged_reads[haplotype] = []
            tagged_reads[haplotype].append(molecule)

    for haplotype in tagged_reads:
        if len(tagged_reads[haplotype]) < 5:
            continue

        with open("{p}_{bc}_cluster{c}_haplotype{h}.whitelist".format(p=plate, bc=barcode, c=i, h=haplotype), "w") as whitelist:
            for molecule in tagged_reads[haplotype]:
                print(molecule, file=whitelist)

In [139]:
# +--------------------------------------------------------------+
# | Run LAA to generate consensus using the haplotype whitelists |
# +--------------------------------------------------------------+
whitelists = !ls *.whitelist
prefixes = ["".join(w.split(".")[:-1]) for w in whitelists]
jobs = ["../../laa.nophase.qsub.sh {p} {s} {p}.whitelist".format(p=p, s=subreads_bam) for p in prefixes]
run_cluster_jobs(jobs)

submitted job ids ['3905909']
job 3905909 finished


In [140]:
# +-------------------------------------------------+
# | Align the amplicon sequences back to the genome |
# +-------------------------------------------------+
laa_outputs = !ls *.laa.nophase.fastq
sequences = list(itertools.chain(r for f in laa_outputs for r in SeqIO.parse(f, "fastq")))
for i,s in enumerate(sequences):
    s.id = laa_outputs[i].split(".")[0] + "_" + s.id.split("_")[-1]
    s.name = s.id
    s.description = s.id
SeqIO.write(sequences, "laa.alleles.fastq", "fastq")

!ngmlr -r {genome} -q laa.alleles.fastq > laa.alleles.aligned.sam
!samtools sort laa.alleles.aligned.sam > laa.alleles.aligned.bam
!samtools index laa.alleles.aligned.bam

ngmlr 0.2.7 (build: Jul  2 2018 10:32:15, start: 2018-10-18.16:02:32)
Contact: philipp.rescheneder@univie.ac.at
Writing output (SAM) to stdout
Reading encoded reference from /bam-export/Guy/GRCh38_no_alt_analysis_set/sequence/GRCh38_no_alt_analysis_set.fasta-enc.2.ngm
Reading 3100 Mbp from disk took 1.06s
Reading reference index from /bam-export/Guy/GRCh38_no_alt_analysis_set/sequence/GRCh38_no_alt_analysis_set.fasta-ht-13-2.2.ngm
Reading from disk took 4.74s
Opening query file laa.alleles.fastq
Mapping reads...
Processed: 1 (1.00), R/S: 0.50, RL: 6646, Time: 10.00 1.00 38.00, Align: 1.00, 309, 0.99
Processed: 1 (1.00), R/S: 0.50, RL: 6646, Time: 10.00 1.00 38.00, Align: 1.00, 309, 0.99
Done (1 reads mapped (100.00%), 0 reads not mapped, 1 lines written)(elapsed: 0m, 0 r/s)
